# Seleccion del Modelo

Vamos a escoger los mejores hiperparámetros para la construcción de un modelo exitoso de manera automática a traves de diversas técnicas

Se hara uso del conjunto de datos de clasificación de aplicaciones con malware para andorid *CICAAGM*.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score

## Funciones auxiliares

In [2]:
# Construcción de una función que realice el particionado completo
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [3]:
# Separar caractersticas de entrada de la etuqieta de salida
def remove_labels(df, label_name):
    X = df.drop(label_name, axis=1)
    y = df[label_name].copy()
    return (X, y)

## 1. Leer el conjunto

In [4]:
df = pd.read_csv('../datasets/TotalFeatures-ISCXFlowMeter.csv')

In [5]:
df

,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,...,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward,calss
0,1020586,668,1641,35692,2276876,52,52,679,1390,53.431138,...,0.0,-1,0.000000e+00,2,4194240,1853440,1640,668,32,benign
1,80794,1,1,75,124,75,124,75,124,75.000000,...,0.0,-1,0.000000e+00,2,0,0,0,1,0,benign
2,998,3,0,187,0,52,-1,83,-1,62.333333,...,0.0,-1,0.000000e+00,4,101888,-1,0,3,32,benign
3,189868,9,9,1448,6200,52,52,706,1390,160.888889,...,0.0,-1,0.000000e+00,2,4194240,2722560,8,9,32,benign
4,110577,4,6,528,1422,52,52,331,1005,132.000000,...,0.0,-1,0.000000e+00,2,155136,31232,5,4,32,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631950,530,1,1,74,334,74,334,74,334,74.000000,...,0.0,-1,0.000000e+00,2,0,0,0,1,0,benign
631951,50240627,23,24,4767,6107,52,52,533,855,207.260870,...,9842879.0,9964749,1.196806e+05,2,317952,107008,11,23,32,GeneralMalware
631952,35471450,1,2,52,104,52,52,52,52,52.000000,...,35300000.0,35290631,0.000000e+00,2,3904,88704,1,1,32,asware
631953,41713629,12,26,1821,18643,40,40,489,1390,151.750000,...,20200000.0,32711382,1.770000e+07,2,227456,2432,23,12,20,benign


## 2. Dividir el conjunto

In [5]:
train_set, val_set, test_set = train_val_test_split(df)

In [6]:
x_train, y_train = remove_labels(train_set, 'calss')
x_val, y_val = remove_labels(val_set, 'calss')
x_test, y_test = remove_labels(test_set, 'calss')

## 3. Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Aplicando de manera manual los hiperparametros
model_rforest = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)
model_rforest.fit(x_train, y_train)

RandomForestClassifier(n_jobs=1, random_state=42)

In [9]:
# Prediccion
y_pred = model_rforest.predict(x_val)

In [10]:
print('Score: ', f1_score(y_pred, y_val, average='weighted'))

Score:  0.9329474731171657


## 4. Seleccion de modelo

Algoritmos complejos como el árbol de decisión requieren de un gran número de hiperparámetros correctamente calibrados para dar un buen resultado, para ello existen diversas técnicas automáticas de selección del modelo.

In [ ]:
# Uso de GridSearch para escoger el modelo
# Para ello creamos un arreglo de diccionarios cuyas claves seran los hiperparametros
# y valores seran un arreglo de combinaciones de valores que estos puedan adoptar.
# Al final se escogera la mejor combinacion automaticamente para crear el mejor modelo posible
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
    { 'n_estimators': [100, 500, 1000], 'max_leaf_nodes': [16, 24, 36] },
    { 'bootstrap': [False], 'n_estimators': [100, 500], 'max_features': [2, 3, 4] }
]

# Preparamos el modelo
model_rforest = RandomForestClassifier(n_jobs=-1, random_state=42)

# Usar GridSearch para obtener los mejores hiperparametros
grid_search = GridSearchCV(model_rforest, param_grid, cv=5, scoring='f1_weighted', return_train_score=True)

grid_search.fit(x_train, y_train)

Los parametros para GridSearch significan:

1. Modelo a entrenar (model_rforest)
2. Arreglo de combinaciones de hiperparametros del modelo a realizar (param_grid)
3. Subconjuntos que se generaran para aplicar a cada combinacion de modelo que creará (cv=5)
4. Tipo de evaluación para determinar la exactitud del modelo (scoring='f1_weighted')

El problema de esta técnica es que requiere de demasiados recursos computacionales y tiempo para ejecutarse por ello solo es útil cuando **no** poseemos una gran cantidad de combinaciones en los hiperparametros.

La alternativa mas eficaz y utilizada es **RandomizedSearchCV** que funciona similar pero realizando una busqueda sobre valores aleatorios.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# En este caso los hipermarametros mas convenientes para nuestro modelo no se obtienen de
# una combinacion de valores si no que es a traves de valores aleatorios entre un rango
# que definimos

param_dist = {
    'n_estimators': randint(low=1, high=200),
    'max_depth': randint(low=8, high=50)
}

model_rforest = RandomForestClassifier(n_jobs=-1)

automodel_rforest = RandomizedSearchCV(
    model_rforest,
    param_distributions=param_dist,
    n_iter=5,
    cv=2,
    scoring='f1_weighted'
)

automodel_rforest.fit(x_train, y_train)

/home/ljmor/anaconda3/envs/curso_deep_learning/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1), n_iter=5,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x758c05d38a10>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x758c05d392e0>},
                   scoring='f1_weighted')

In [10]:
# Podemos visualizar cuales son los mejores hiperparametros que escogio el algoritmo
automodel_rforest.best_params_

{'max_depth': 45, 'n_estimators': 190}

In [11]:
# Cual es el mejor modelo listo para ser utilizado
automodel_rforest.best_estimator_

RandomForestClassifier(max_depth=45, n_estimators=190, n_jobs=-1)

In [12]:
# Comparar la exactitud de las diversas combinaciones de hiperparametros
cvres = automodel_rforest.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print('Score: ', mean_score, '-', 'Parametros: ', params)

Score:  0.9244362266321879 - Parametros:  {'max_depth': 46, 'n_estimators': 30}
Score:  0.926197619534796 - Parametros:  {'max_depth': 45, 'n_estimators': 190}
Score:  0.8551864979842885 - Parametros:  {'max_depth': 8, 'n_estimators': 53}
Score:  0.9224591571207529 - Parametros:  {'max_depth': 18, 'n_estimators': 61}
Score:  0.9259720515723482 - Parametros:  {'max_depth': 45, 'n_estimators': 154}


## 5. Modelo Final

Hacemos uso del modelo definitivo que ya nos lo provee nuestra tecnica de seleccion automatica de seleccion de hiperparametros

In [13]:
automodel_rforest.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 45,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 190,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [14]:
# Seleccionar el mejor modelo
model_rforest = automodel_rforest.best_estimator_

In [15]:
# Predecir con el conjunto de entrenamiento
y_train_pred = model_rforest.predict(x_train)

In [17]:
f_score = f1_score(y_train_pred, y_train, average='weighted')
print(f'Exactitud para el conjunto de pruebas: {round(f_score * 100, ndigits=2)} %')

Exactitud para el conjunto de pruebas: 98.13 %


In [18]:
# Predecir con el conjunto de validacion
y_val_pred = model_rforest.predict(x_val)

In [19]:
f_score = f1_score(y_val_pred, y_val, average='weighted')
print(f'Exactitud para el conjunto de validacion: {round(f_score * 100, ndigits=2)} %')

Exactitud para el conjunto de validacion: 93.26 %


Así es como habremos obtenido un modelo entrenado con los mejores hiperparametros que le podemos dar para que funcione de la mejor manera posible